# ヒグマ出没データのCSVデータをもとに、geojson形式に変換するノートブック
## データ出典
[札幌市オープンデータ「熊の出没情報」](https://ckan.pf-sapporo.jp/dataset/sapporo_bear_appearance)

当該データは [クリエイティブ・コモンズ 表示 4.0 国際ライセンス（CC BY 4.0）](https://creativecommons.org/licenses/by/4.0/deed.ja) の下で提供されています。

© 札幌市, CC BY 4.0

## properties項目定義

CSVの内容から、propertiesに以下を設定します。
- id: 年と連番をハイフンつなぎ
- datetime: 日付と時刻から YYYY-MM-DD hh:mm 書式の文字列。ただし、時刻が「不明」の場合は YYYY-MM-DD のみ
- year: 日付の年
- month: 日付の月
- hour: 時刻から。ただし、時刻が入っていない場合は空欄
- award: 区
- location: 出没場所
- status: 状況
- icon: アイコンファイル名

# 定数定義

In [ ]:
CSV_DIR = './data/csv'
GEOJSON_DIR = './data/geojson'
GEOJSON_FILE = 'bears.geojson'

# 必要ライブラリのインポート

In [ ]:
import os
import datetime
import traceback

import pandas as pd
from geojson import Point, Feature, FeatureCollection, dump

# CSV読み取り

In [ ]:
csv_files: list[str] = []
files = os.listdir(CSV_DIR)
for file in files:
    if file.endswith('.csv'):
        csv_files.append(file)

In [ ]:
csv_files.sort()

In [ ]:
csv_files

In [ ]:
df: pd.DataFrame = pd.DataFrame()
for file in csv_files:
    tmp = pd.read_csv('{}/{}'.format(CSV_DIR, file,), encoding='utf8', parse_dates=['日付',])
    df = pd.concat([df,tmp]) 

In [ ]:
df

# フィルタ用項目を追加

In [ ]:
df.dtypes

In [ ]:
df['year'] = df['日付'].dt.year
df['month'] = df['日付'].dt.month

In [ ]:
max_index_len = len(str(max(df.index)))
# max_index_len

In [ ]:
id_list: list[str] = []
datetime_list: list[datetime.datetime] = []
sequence = 0
for idx, row in df.iterrows():
    # id_list.append('{year}-{idx}'.format(year = row['year'], idx = str(idx).zfill(max_index_len),))
    sequence += 1
    id_list.append(str(sequence).zfill(max_index_len))

    try:
        datetime_list.append(datetime.datetime.strptime(row['日付'].strftime('%Y-%m-%d') + ' ' + row['時刻'], '%Y-%m-%d %H:%M'))
    except Exception as exc:
        print('----- 以下の行で例外が発生しました -----')
        print(row)
        print('')
        print('例外内容：')
        print(exc)
        print('')
        # print('例外詳細：')
        # print(''.join(traceback.TracebackException.from_exception(exc).format()))
        # print('')
        print('処理内容：')
        # print("時刻を 0:00 として生成しました")
        # datetime_list.append(datetime.datetime.strptime(row['日付'].strftime('%Y-%m-%d') + ' 0:00', '%Y-%m-%d %H:%M'))
        row_date = row['日付'].date()
        print('日付 {date} のみ登録しました ※type：{type}'.format(date = row_date.strftime('%Y-%m-%d'), type = type(row_date),))
        datetime_list.append(row_date)
        print('')
        print('')
df['id'] = id_list
df['datetime'] = datetime_list

In [ ]:
hour_list: list[int] = []
for i, row in df.iterrows():
    if type(row['datetime']) is datetime.datetime:
         hour_list.append(row['datetime'].hour)
    else:
        hour_list.append('')
df['hour'] = hour_list    

In [ ]:
def judge_icon(row):
    status = row['状況']

    if status.startswith('ヒグマを目撃'): # 101
        return 'bear'
    elif 'ヒグマを確認' in status: # 102
        return 'bear'
    elif status.startswith('ヒグマを駆除'): # 103
        return 'bear'
    elif 'ヒグマを捕獲' in status: # 104
        return 'bear'
    elif status.startswith('ヒグマ動物を目撃'): # 105
        return 'bear'
    elif status.startswith('ヒグマらしき動物を目撃'): # 201
        return 'like-bear'
    elif 'フン' in status: # 301
        return 'excrement'
    elif '足跡' in status: # 401
        return 'footprint'
    elif 'カメラ' in status: # 501
        return 'camera'
    elif '声' in status: # 601
        return 'voice'
    elif '音' in status: # 602
        return 'voice'
    else: # 9999
        return 'other'

In [ ]:
df['icon'] = df.apply(judge_icon, axis=1)

In [ ]:
df

# geojson生成

参考：
```python
from geojson import Point, Feature, FeatureCollection, dump
outfile = "out.geojson"

PointsData = [{'name': 'Ho Chi Minh City Opera House',
               'latitude': 10.777406835725145,
               'longitude': 106.70299858740313,
               'color': '#ff2600'},
              {'name': 'Independence Palace',
               'latitude': 10.778137451508647,
               'longitude': 106.69531332149265,
               'color': '#00ff26'}]

# Prepare Geojson FeatureCollection
ft_all = []
for i, p in enumerate(PointsData):
    lat, lon = p['latitude'], p['longitude']
    ft = Feature(geometry = Point((lon, lat,)),
                 properties = {'name': p['name'], 'marker-color': p['color'],
                               'marker-size': 'medium','marker-symbol': ''})
    ft_all.append(ft)
ft_colct = FeatureCollection(ft_all)

with open(outfile, 'w') as f:
    dump(ft_colct, f, indent=2)

```

In [ ]:
ft_all = []
for i, row in df.iterrows():
    row_info = row['id'], row['datetime'].strftime('%Y-%m-%d %H:%M')

    try:
        date_s: str = None
        if type(row['datetime']) is datetime.datetime:
            date_s = row['datetime'].strftime('%Y年%m月%d日 %H:%M')
        elif type(row['datetime']) is datetime.date:
            date_s = row['datetime'].strftime('%Y年%m月%d日')

        point = Point((float(row['経度']), float(row['緯度']),))
        properties = {
            'id': row['id'],
            'datetime': date_s,
            'year': row['year'],
            'month': row['month'],
            'hour': row['hour'],
            'award': row['区'],
            'location': row['出没場所'],
            'status': row['状況'],
            'icon': row['icon'] + '.svg'
        }
    
        ft = Feature(geometry = point, properties = properties,)
    
        ft_all.append(ft)

    except Exception as exc:
        print('----- 以下の行で例外が発生しました -----')
        print(row)
        print('')
        print('例外内容：')
        print(''.join(traceback.TracebackException.from_exception(exc).format()))

In [ ]:
len(ft_all)

In [ ]:
for row in ft_all[len(ft_all) - 5:]:
    print(row)

In [ ]:
ft_colct = FeatureCollection(ft_all)

In [ ]:
outfile = '{}/{}'.format(GEOJSON_DIR, GEOJSON_FILE,)
with open(outfile, 'w') as f:
    dump(ft_colct, f, indent=2, ensure_ascii=False)